In [ ]:
# default_exp core

# Simple Neo4j HTTP API Client

> Simple http client to connect neo4j server

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os,json,base64
import requests
import pandas as pd
from neo4jtools.dwpc import make_dwpc_query
from neo4jtools.utils import graph_renderer, row_renderer
from neo4jtools.graph import draw

class SimpleNeo4jHTTPAPIClient:
    
    def __init__(self, url, db='neo4j', userid=None, passwd=None):
        self.url=url
        self.set_serverinfo(url)
        self.setdb(db)
        self.authtoken=None
        if userid is not None:
            self.authtoken=self.get_authtoken(userid, passwd)
        
    def set_serverinfo(self,url):
        resp=requests.get(url)
        obj=resp.json()
        self.bolt_routing=obj['bolt_routing']
        self.transaction=obj['transaction']
        self.bolt_direct=obj['bolt_direct']
        self.neo4j_version=obj['neo4j_version']
        self.neo4j_edition=obj['neo4j_edition']

    def setdb(self, db):
        self.db=db
    
    def execute_read_query(self, query, output_format=['row','graph']):
        return execute_query(self, query, output_format)
    
    def execute_query(self, query, output_format=['row','graph']):
        url=self.transaction.format(databaseName=self.db) + '/commit'
        headers={
            "content-type": "application/json"
        }
        if self.authtoken is not None:
            headers['authorization']="Basic {}".format(self.authtoken)

        if isinstance(output_format, list):
            resultDataContents = output_format
        else:
            resultDataContents = [output_format]

        statement={
          "statements": [
            {
              "statement": query,
              "resultDataContents": resultDataContents
            }
          ]
        }
        resp=requests.post(url, 
                           data=json.dumps(statement), 
                           headers=headers)

        output= resp.json()

        if len(output['errors']) > 0: 
            raise Exception(output['errors'])
            
        
        output_renderer=None
        if output_format=='row':
            output_renderer=row_renderer
                    
        if output_format=='graph':
            output_renderer=graph_renderer
        
        if output_renderer:
            output=output_renderer(output)
        
        return output
    
    def show(self, query):
        graph= self.execute_read_query(query, output_format='graph')
        return draw([graph])
    
    def calculate_dwpc(self, genes, reltype, hops, 
                       dwpc_score_prop_name='dwpc_score', 
                       only_relations_with_pmid=True,
                       debug=False):
        qry=make_dwpc_query(genes, 
                            reltype=reltype, 
                            hops=hops, 
                            dwpc_score_prop_name=dwpc_score_prop_name, 
                            only_relations_with_pmid=only_relations_with_pmid)
        if debug:
            print(qry)
            
        results=self.execute_read_query(qry, output_format='row')
        
        return results
    
    def __repr__(self):
        return json.dumps({
            'classname':self.__class__.__name__,
            'url':self.url,
            'neo4j_version':self.neo4j_version,
            'neo4j_edition':self.neo4j_edition,
            'db':self.db,
            'auth': self.authtoken is not None
        })

    @staticmethod
    def get_authtoken(userid, passwd):
        authstr='{}:{}'.format(userid, passwd)
        b64token=base64.b64encode(authstr.encode())
        strtoken=b64token.decode()
        return strtoken

## Simple example

In [ ]:
# Connect to a neo4j server without auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.3", "neo4j_edition": "community", "db": "neo4j", "auth": false}

In [ ]:
try:
    client.execute_read_query('match (n) return count(n);', output_format=['row'])
except Exception as e:
    assert e

In [ ]:
# Connect to a neo4j server with auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474', userid='neo4j',passwd='test')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.3", "neo4j_edition": "community", "db": "neo4j", "auth": true}

In [ ]:
result=client.execute_read_query('match (n) return count(n);')
result

{'results': [{'columns': ['count(n)'],
   'data': [{'row': [85852],
     'meta': [None],
     'graph': {'nodes': [], 'relationships': []}}]}],
 'errors': []}

## `row_renderer` (output_format='row') example

In [ ]:
result=client.execute_read_query('match p=(n)-[r]-(m) return n.identifier, n.name, type(r), m.identifier, m.name limit 3;', output_format='row')
result

[{'n.identifier': 'GO:0000002',
  'n.name': 'mitochondrial genome maintenance',
  'type(r)': 'PARTICIPATES_GpBP',
  'm.identifier': '4358',
  'm.name': 'MPV17'},
 {'n.identifier': 'GO:0000002',
  'n.name': 'mitochondrial genome maintenance',
  'type(r)': 'PARTICIPATES_GpBP',
  'm.identifier': '291',
  'm.name': 'SLC25A4'},
 {'n.identifier': 'GO:0000002',
  'n.name': 'mitochondrial genome maintenance',
  'type(r)': 'PARTICIPATES_GpBP',
  'm.identifier': '55186',
  'm.name': 'SLC25A36'}]

In [ ]:
pd.DataFrame(result)

,n.identifier,n.name,type(r),m.identifier,m.name
0,GO:0000002,mitochondrial genome maintenance,PARTICIPATES_GpBP,4358,MPV17
1,GO:0000002,mitochondrial genome maintenance,PARTICIPATES_GpBP,291,SLC25A4
2,GO:0000002,mitochondrial genome maintenance,PARTICIPATES_GpBP,55186,SLC25A36


## `graph_renderer` (output_format='graph') example

In [ ]:
result=client.execute_read_query('match p=()--() return p limit 3;', output_format='graph')
result

{'nodes': [{'identifier': '4358',
   'name': 'MPV17',
   'label': 'Gene',
   'license': 'CCO 1.0',
   'source': 'Entrez Gene:210321',
   'minkyu_degree': 14.0,
   'url': 'http://identifiers.org/ncbigene/4358',
   'description': 'mitochondrial inner membrane protein MPV17',
   'chromosome': '2'},
  {'identifier': 'GO:0000002',
   'name': 'mitochondrial genome maintenance',
   'label': 'BiologicalProcess',
   'license': 'CC BY 4.0',
   'source': 'Gene Ontology:2021-02-01',
   'minkyu_degree': nan,
   'url': 'http://purl.obolibrary.org/obo/GO:0000002',
   'description': nan,
   'chromosome': nan},
  {'identifier': '291',
   'name': 'SLC25A4',
   'label': 'Gene',
   'license': 'CCO 1.0',
   'source': 'Entrez Gene:210321',
   'minkyu_degree': 123.0,
   'url': 'http://identifiers.org/ncbigene/291',
   'description': 'solute carrier family 25 member 4',
   'chromosome': '4'},
  {'identifier': '55186',
   'name': 'SLC25A36',
   'label': 'Gene',
   'license': 'CCO 1.0',
   'source': 'Entrez Gen

In [ ]:
assert 'nodes' in result and 'edges' in result

In [ ]:
pd.DataFrame(result['nodes'])

,identifier,name,label,license,source,minkyu_degree,url,description,chromosome
0,4358,MPV17,Gene,CCO 1.0,Entrez Gene:210321,14.0,http://identifiers.org/ncbigene/4358,mitochondrial inner membrane protein MPV17,2
1,GO:0000002,mitochondrial genome maintenance,BiologicalProcess,CC BY 4.0,Gene Ontology:2021-02-01,NaN,http://purl.obolibrary.org/obo/GO:0000002,NaN,NaN
2,291,SLC25A4,Gene,CCO 1.0,Entrez Gene:210321,123.0,http://identifiers.org/ncbigene/291,solute carrier family 25 member 4,4
3,55186,SLC25A36,Gene,CCO 1.0,Entrez Gene:210321,2.0,http://identifiers.org/ncbigene/55186,solute carrier family 25 member 36,3


In [ ]:
pd.DataFrame(result['edges'])

,type,start_identifier,start_name,end_identifier,end_name,license,unbiased,source,version
0,PARTICIPATES_GpBP,4358,MPV17,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,NCBI gene2go,2021-02-01
1,PARTICIPATES_GpBP,291,SLC25A4,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,NCBI gene2go,2021-02-01
2,PARTICIPATES_GpBP,55186,SLC25A36,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,NCBI gene2go,2021-02-01


## `show` example

In [ ]:
query="MATCH p0=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'})  MATCH p1=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'8313'})-[:INTERACTS_GiG3]->(:Gene{identifier:'1499'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'})  MATCH p2=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'4609'})-[:INTERACTS_GiG3]->(:Gene{identifier:'22943'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'})  MATCH p3=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'4609'})-[:INTERACTS_GiG3]->(:Gene{identifier:'4353'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'}) RETURN p0,p1,p2,p3"
query

"MATCH p0=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'})  MATCH p1=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'8313'})-[:INTERACTS_GiG3]->(:Gene{identifier:'1499'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'})  MATCH p2=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'4609'})-[:INTERACTS_GiG3]->(:Gene{identifier:'22943'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'})  MATCH p3=(:Gene{identifier:'6932'})-[:INTERACTS_GiG3]->(:Gene{identifier:'4609'})-[:INTERACTS_GiG3]->(:Gene{identifier:'4353'})-[:INTERACTS_GiG3]->(:Gene{identifier:'51176'}) RETURN p0,p1,p2,p3"

In [ ]:
client.show(query)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…